In [ ]:
import pandas as pd
from fuzzywuzzy import process


import os
os.chdir("../")

EXCEL = "./data/Jury/Bewertung/Master Sheet/Master Sheet.xlsx"
JSON = "./data/texts.json"
COMBINED = "./data/combined.json"

In [ ]:
df2 = pd.read_excel(EXCEL, header=1)
df2['Startup'] = df2['Startup'].str.split(" -").str.get(0)

In [ ]:
df1 = pd.read_json(JSON)
df1.head()

In [ ]:
# Function to find the best match using fuzzy matching
def fuzzy_match(target, choices, threshold=80):
    match, score = process.extractOne(target, choices)
    if score >= threshold:
        return match
    return None

# Extract relevant identifiers from `Filename` (without file extension)
df1['SearchKey'] = df1['Filename'].str.replace('.pdf', '', regex=False)

# Apply fuzzy matching to df1['SearchKey'] against all df2['Startup']
df1['BestMatch'] = df1['SearchKey'].apply(
    lambda x: fuzzy_match(x, df2['Startup'].tolist())
)

# Merge DataFrames based on the fuzzy match result
df = df1.merge(df2, left_on='BestMatch', right_on='Startup', how='left')

In [ ]:
df = df.dropna(subset=['Startup'])
df = df.dropna(subset=['Filename'])

df.to_json(COMBINED, orient='records')